In [9]:
import torch
import torch.nn as nn


# ======== This network is for data of dimension 100x87 (4 s) =================
class Encoder(nn.Module):
    """
    Description: Encoder layers of autoencoder model; encodes input data
    (spectrograms) into the latent feature space.
    Inputs:
        - Input data (spectrograms)
    Outputs:
        - Latent feature space data
    """
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
            nn.ReLU(True),
            nn.Conv2d(8, 16, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
            nn.ReLU(True),
            nn.Conv2d(16, 32, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
            nn.ReLU(True),
            nn.Conv2d(32, 64, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=(3,3), stride=(2,2), padding=(1,0)),
            nn.ReLU(True),
            nn.Flatten(),
            nn.Linear(1152, 9),
            nn.ReLU(True)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x


class Decoder(nn.Module):
    """
    Description: Decoder layers of autoencoder model; reconstructs encoder
    inputs using the latent features generated by the encoder.
    Inputs:
        - Latent space data
    Outputs:
        - Reconstructed data
    """
    def __init__(self):
        super(Decoder, self).__init__()
        self.latent2dec = nn.Sequential(
            nn.Linear(9, 1152),
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=(3,3), stride=(2,2), padding=(1,0)), # <---- Experimental
            nn.ReLU(True),  # <---- Experimental
            nn.ConvTranspose2d(64, 32, kernel_size=(3,3), stride=(2,2), padding=(0,1)),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, kernel_size=(3,3), stride=(2,2), padding=(0,1)),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, kernel_size=(3,3), stride=(2,2), padding=(0,0)),
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, kernel_size=(3,3), stride=(2,2), padding=(0,1)),
        )

    def forward(self, x):
        x = self.latent2dec(x)
        x = x.view(-1, 128, 3, 3)
        x = self.decoder(x)
        return x[:,:,4:-4,1:]

class AEC(nn.Module):
    """
    Description: Autoencoder model; combines encoder and decoder layers.
    Inputs:
        - Input data (spectrograms)
    Outputs:
        - Reconstructed data
        - Latent space data
    """
    def __init__(self):
        super(AEC, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        z = self.encoder(x)
        x = self.decoder(z)
        return x, z

    

In [10]:
%reload_ext autoreload
%autoreload 2
from data import *
from train import *
from networks import *
from visualisation import *
VALIDATION_DATASET_PERC = 0.2
TEST_DATASET_PERC = 0.1

dataset = H5SeismicDataset(
    filepath="/datasets/ee228-sp21-A00-public/RIS_Seismic-001.h5",
    transform = transforms.Compose(
        [SpecgramShaper(), SpecgramToTensor()]
    )
)
#dataset[0]
dataloaders = getDataloaderSplit(dataset, VALIDATION_DATASET_PERC,TEST_DATASET_PERC)

In [11]:
model= AEC()

In [12]:
outputs = pretrain(model= model,dataloaders=dataloaders,device=device)

NameError: name 'device' is not defined

In [13]:
if not torch.cuda.is_available():
    print('No cuda')
#params= {'lr':0.01,'epochs':50}
device = torch.device("cuda")
model= AEC()
PATH_WEIGHTS='./saved_models/AEC_weights'
torch.save(model.state_dict(),PATH_WEIGHTS)
model.to(device)
outputs = pretrain(model= model,dataloaders=dataloaders,device=device)
PATH='./saved_models/AEC'
torch.save(model, PATH)

epoch:   0  train loss:  tensor(0.0380)
epoch:   0  train loss:  tensor(0.0239)
epoch:   0  train loss:  tensor(0.0246)
epoch:   0  train loss:  tensor(0.0250)
epoch:   0  train loss:  tensor(0.0252)
epoch:   0  train loss:  tensor(0.0255)
epoch:   0  val loss:  tensor(0.0239)
epoch:   0  val loss:  tensor(0.0242)
epoch:   0  val loss:  tensor(0.0248)

train loss tensor(0.0246) val_loss  tensor(0.0245)

epoch:   1  train loss:  tensor(0.0249)
epoch:   1  train loss:  tensor(0.0334)
epoch:   1  train loss:  tensor(0.0318)
epoch:   1  train loss:  tensor(0.0246)
epoch:   1  train loss:  tensor(0.0417)
epoch:   1  train loss:  tensor(0.0230)
epoch:   1  val loss:  tensor(0.0447)
epoch:   1  val loss:  tensor(0.0450)
epoch:   1  val loss:  tensor(0.0457)

train loss tensor(0.0308) val_loss  tensor(0.0448)

epoch:   2  train loss:  tensor(0.0449)


KeyboardInterrupt: 